In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from sklearn.tree import DecisionTreeClassifier
import graphviz
from random import * 
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score,GridSearchCV,cross_val_predict

In [2]:
data = pd.read_csv("../classificador/Saidas/features_random_forest.csv")
data = data.apply(lambda x: x.replace("madrugada", 0))
data = data.apply(lambda x: x.replace("manha", 1))
data = data.apply(lambda x: x.replace("tarde", 2))
data = data.apply(lambda x: x.replace("noite", 3))

data = data.apply(lambda x: x.replace(True, 1))
data = data.apply(lambda x: x.replace(False, 0))

#data['Horario'] = pd.to_datetime(data['Horario'], format="%H:%M:%S")
#data['Horario'] = (data['Horario'] - dt.datetime(1900,1,1)).dt.total_seconds()

data.drop(data.columns[0:2], axis=1,inplace=True)
data.drop(data.columns[2], axis=1,inplace=True)
data.drop(data.columns[4], axis=1,inplace=True)
data.drop(data.columns[-1:], axis=1,inplace=True)

data.drop(data.index[1274:],inplace=True)

In [3]:
data

,Hora,Minuto,Fim de Semana,Sensor Comodo,lig_desl_luz,lig_desl_aparelho,tempo_lig_luz,tempo_lig_aparelho,Id Atividade
0,13,28,1.0,16,0,0,1.0,0.000000,5
1,13,28,1.0,15,0,0,1.0,0.000000,6
2,13,32,1.0,16,0,0,1.0,0.000000,5
3,13,32,1.0,15,0,0,1.0,0.000000,6
4,13,44,1.0,16,0,0,1.0,0.000000,5
...,...,...,...,...,...,...,...,...,...
1269,19,39,0.0,16,0,1,1.0,0.769231,5
1270,19,39,0.0,15,0,0,1.0,1.000000,6
1271,20,23,0.0,15,0,0,1.0,1.000000,6
1272,21,4,0.0,16,0,0,1.0,0.000000,5


In [4]:
#label_encoder = preprocessing.LabelEncoder()
#data['Horario'] = label_encoder.fit_transform(data['Horario'])

In [5]:
j = 0
lista_indice = []
while True:
	num = randint(0,len(data) - 1)
	add =True
	for element in lista_indice:
		if num == element:
			add = False
	if add:
		lista_indice.append(num)
	if len(lista_indice) == 50:
		break

print(lista_indice)

[1144, 100, 842, 163, 434, 564, 905, 1263, 883, 7, 813, 965, 121, 265, 864, 544, 204, 214, 478, 484, 12, 1058, 907, 503, 677, 416, 715, 963, 425, 420, 772, 280, 538, 1221, 149, 169, 232, 980, 897, 480, 333, 162, 358, 385, 1270, 8, 1123, 1271, 845, 473]


In [6]:
val = pd.DataFrame(columns = data.columns)
j = 0
for element in lista_indice:
	val.loc[j] = data.loc[element]
	data.drop([element],inplace=True)
	j += 1
data.reset_index(drop=True, inplace =True)

In [50]:
################# data train and test_x #################################################

features = list(data.columns[0:-1])
target = data.columns[-1:]

In [51]:
train_x = data[features]	# train data
train_y = data[target] 		# train data
test_x = val[features] 		# test data 
test_y = val[target]		# test data

df_test2 = pd.DataFrame(columns = data.columns)
#df_test2.loc[0] = [(dt.datetime(1900,1,1,6,15,0) - dt.datetime(1900,1,1)).total_seconds(),1.0,17,0,0,0.0,0.0,2]
df_test2.loc[0] = [6,30,1.0,17,0,0,0.0,0.0,2]
test2x = df_test2[features]
test2y = df_test2[target]

In [52]:
train_x, train_y

(      Hora  Minuto  Fim de Semana  Sensor Comodo  lig_desl_luz  \
 0       13      28            1.0             16             0   
 1       13      28            1.0             15             0   
 2       13      32            1.0             16             0   
 3       13      32            1.0             15             0   
 4       13      44            1.0             16             0   
 ...    ...     ...            ...            ...           ...   
 1219    19      30            0.0             15             0   
 1220    19      36            0.0             15             0   
 1221    19      39            0.0             16             0   
 1222    21       4            0.0             16             0   
 1223    21       4            0.0             15             0   
 
       lig_desl_aparelho  tempo_lig_luz  tempo_lig_aparelho  
 0                     0            1.0            0.000000  
 1                     0            1.0            0.000000  
 2      

In [53]:
test2x, test2y

(   Hora  Minuto  Fim de Semana  Sensor Comodo  lig_desl_luz  \
 0   6.0    30.0            1.0           17.0           0.0   
 
    lig_desl_aparelho  tempo_lig_luz  tempo_lig_aparelho  
 0                0.0            0.0                 0.0  ,    Id Atividade
 0           2.0)

In [54]:
########################################### tree ########################################
clf_tree = DecisionTreeClassifier()
#clf_tree.fit(train_x,train_y)

grid_tree = GridSearchCV(clf_tree, {}, cv=10)
grid_tree.fit(train_x, train_y)
tree = DecisionTreeClassifier().fit(train_x,train_y)
tree_cross = grid_tree.best_estimator_

print(tree_cross.score(test2x,test2y), tree.score(test2x,test2y))

#result = tree.predict(test_x)

1.0 1.0


In [55]:
###################################### forest #########################################
clf_forest = RandomForestClassifier()
train_y = np.reshape(train_y.to_numpy(),-1)

param_grid = {
                 'n_estimators': [5, 10, 15, 20],
  				 'max_depth': [2, 5, 7, 9]
  			}

grid_forest = GridSearchCV(clf_forest, param_grid, cv=10)
grid_forest.fit(train_x, train_y)
forest = RandomForestClassifier().fit(train_x,train_y)
forest_cross = grid_forest.best_estimator_
print(forest_cross.score(test2x.values, test2y.values), forest.score(test2x.values, test2y.values))



#forest = RandomForestClassifier(n_estimators=100, max_depth=4,random_state=0)
#forest.fit(train_x, train_y)
#print(forest.predict_proba(test_x.values),forest.predict([[0.3,0.6,0.7,0.5]]))

0.0 0.0


c:\users\victor\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
c:\users\victor\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
